# Call Context Summary

In [44]:
import json
import pathlib
import copy

In [45]:
data_dir = pathlib.Path.cwd().parent / "data"
trace_file = data_dir / "fibonacci" / "paptrace.json"
with open(trace_file, "r") as f_in:
    traces = json.load(f_in)["traces"]

In [46]:
def populate_sig_map(tree, sig_map):
    if isinstance(tree, list):
        for item in tree:
            populate_sig_map(item, sig_map)
    elif isinstance(tree, dict):
        if "sig" in tree:
            k = tree["sig"]
            if k not in sig_map:
                sig_map[k] = len(sig_map)
        for key, value in tree.items():
            populate_sig_map(value, sig_map)

sig_map = {}
populate_sig_map(traces, sig_map)
print(f"Number of unique signatures: {len(sig_map)}")
display(sig_map)

Number of unique signatures: 6


{'unsigned long long fibonacci::RecursiveNaive(unsigned short)': 0,
 'unsigned long long fibonacci::RecursiveMemo(unsigned short)': 1,
 'reference std::vector<unsigned long long>::operator[](size_type)': 2,
 'unsigned long long fibonacci::(anonymous namespace)::RecursiveMemoImpl(unsigned short, std::vector<unsigned long long> &)': 3,
 'unsigned long long fibonacci::Iterative(unsigned short)': 4,
 'unsigned long long fibonacci::LookupTable(unsigned short)': 5}

In [47]:
def extract_call_nodes(tree, sig_map, call_nodes):
    if isinstance(tree, list):
        for item in tree:
            extract_call_nodes(item, sig_map, call_nodes)
    elif isinstance(tree, dict):
        for key, value in tree.items():
            extract_call_nodes(value, sig_map, call_nodes)
        if "type" in tree and "Call" in tree["type"]:
            tree["id"] = sig_map[tree["sig"]]
            call_nodes.append(copy.deepcopy(tree))
            tree.pop("children")

call_nodes = []
extract_call_nodes(traces, sig_map, call_nodes)
print(f"Number of call nodes: {len(call_nodes)}")
display(call_nodes[:3])

Number of call nodes: 288


[{'children': [{'children': [{'children': [],
      'desc': 'return n',
      'id': 2106007,
      'type': 'ReturnStmt'}],
    'desc': 'n < 2',
    'id': 2106009,
    'type': 'IfThenStmt'}],
  'id': 0,
  'params': [{'name': 'n', 'value': '0'}],
  'sig': 'unsigned long long fibonacci::RecursiveNaive(unsigned short)',
  'type': 'CalleeExpr'},
 {'children': [{'children': [{'children': [],
      'desc': 'return n',
      'id': 2106007,
      'type': 'ReturnStmt'}],
    'desc': 'n < 2',
    'id': 2106009,
    'type': 'IfThenStmt'}],
  'id': 0,
  'params': [{'name': 'n', 'value': '1'}],
  'sig': 'unsigned long long fibonacci::RecursiveNaive(unsigned short)',
  'type': 'CalleeExpr'},
 {'children': [{'children': [{'children': [],
      'desc': 'return n',
      'id': 2106007,
      'type': 'ReturnStmt'}],
    'desc': 'n < 2',
    'id': 2106009,
    'type': 'IfThenStmt'}],
  'id': 0,
  'params': [{'name': 'n', 'value': '1'}],
  'sig': 'unsigned long long fibonacci::RecursiveNaive(unsigned short

In [48]:
unique_call_nodes = []
unique_call_node_strs = {}
for call_node in call_nodes:
    call_node_str = str(call_node)
    if not call_node_str in unique_call_node_strs:
        unique_call_node_strs[call_node_str] = 1
        unique_call_nodes.append(call_node)

print(f"Number of uniq. call nodes: {len(unique_call_nodes)}")
display(unique_call_nodes[:3])

Number of uniq. call nodes: 162


[{'children': [{'children': [{'children': [],
      'desc': 'return n',
      'id': 2106007,
      'type': 'ReturnStmt'}],
    'desc': 'n < 2',
    'id': 2106009,
    'type': 'IfThenStmt'}],
  'id': 0,
  'params': [{'name': 'n', 'value': '0'}],
  'sig': 'unsigned long long fibonacci::RecursiveNaive(unsigned short)',
  'type': 'CalleeExpr'},
 {'children': [{'children': [{'children': [],
      'desc': 'return n',
      'id': 2106007,
      'type': 'ReturnStmt'}],
    'desc': 'n < 2',
    'id': 2106009,
    'type': 'IfThenStmt'}],
  'id': 0,
  'params': [{'name': 'n', 'value': '1'}],
  'sig': 'unsigned long long fibonacci::RecursiveNaive(unsigned short)',
  'type': 'CalleeExpr'},
 {'children': [{'children': [{'id': 0,
      'params': [{'name': 'n', 'value': '1'}],
      'sig': 'unsigned long long fibonacci::RecursiveNaive(unsigned short)',
      'type': 'CalleeExpr'},
     {'id': 0,
      'params': [{'name': 'n', 'value': '0'}],
      'sig': 'unsigned long long fibonacci::RecursiveNaive(u

In [49]:
call_map = {}
for call_node in unique_call_nodes:
    id = call_node["id"]
    if not id in call_map:
        call_map[id] = {"sig": call_node["sig"], "ctxs": {}}
    ctx = str(call_node["params"])
    if not ctx in call_map[id]["ctxs"]:
        call_map[id]["ctxs"][ctx] = len(call_map[id]["ctxs"])
    else:
        raise Exception(f"Duplicate call node ctx: {ctx}!")
        
print(f"Number of top-level call map entries: {len(call_map)}")
display(call_map)

Number of top-level call map entries: 6


{0: {'sig': 'unsigned long long fibonacci::RecursiveNaive(unsigned short)',
  'ctxs': {"[{'name': 'n', 'value': '0'}]": 0,
   "[{'name': 'n', 'value': '1'}]": 1,
   "[{'name': 'n', 'value': '2'}]": 2,
   "[{'name': 'n', 'value': '3'}]": 3,
   "[{'name': 'n', 'value': '4'}]": 4,
   "[{'name': 'n', 'value': '5'}]": 5,
   "[{'name': 'n', 'value': '6'}]": 6,
   "[{'name': 'n', 'value': '7'}]": 7,
   "[{'name': 'n', 'value': '94'}]": 8}},
 2: {'sig': 'reference std::vector<unsigned long long>::operator[](size_type)',
  'ctxs': {"[{'name': 'memo', 'value': '{ 0 }'}, {'name': '1', 'value': '1'}]": 0,
   "[{'name': 'memo', 'value': '{ 0 }'}, {'name': 'n', 'value': '0'}]": 1,
   "[{'name': 'memo', 'value': '{ 0, 0 }'}, {'name': '1', 'value': '1'}]": 2,
   "[{'name': 'memo', 'value': '{ 0, 1 }'}, {'name': 'n', 'value': '1'}]": 3,
   "[{'name': 'memo', 'value': '{ 0, 0, 0 }'}, {'name': '1', 'value': '1'}]": 4,
   "[{'name': 'memo', 'value': '{ 0, 1, 0 }'}, {'name': 'n', 'value': '2'}]": 5,
   "[{